In [18]:
import os
import glob
import numpy as np
import pandas as pd
from tifffile import imread
import h5py
import matplotlib.pyplot as plt
from scripts.sample_db import SampleDB

# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)


In [19]:

# Loading experiment
sample_id = '20220118_RM0012_124hpf_fP8_f2'  
exp = sample_db.get_sample(sample_id)
print(exp.sample.id)

# Making shortcuts of sample parameters/information
sample = exp.sample
trials_path = exp.paths.trials_path
n_planes = exp.params_lm.n_planes
n_trials = exp.params_lm.n_trials
doubling = 2 if exp.params_lm.doubling else 1

# Getting paths of the trial acquisitions
trial_names = os.listdir(os.path.join(trials_path, 'raw'))
processed_folder = os.path.join(trials_path, 'processed')
masks_folder = os.path.join(trials_path, "masks")

# Load the masks
masks_file = glob.glob(os.path.join(masks_folder, f'masks_{exp.sample.id}_*.tif'))[0]
masks_stack = imread(masks_file)

# Create folder for saving fluorescence data
traces_folder = os.path.join(trials_path, "traces")
os.makedirs(traces_folder, exist_ok=True)

hdf5_file_path = os.path.join(traces_folder, f'{exp.sample.id}_fluorescence_data.h5')

# Function to process and save data
def extract_fluorescence_data(hdf5_file_path, sample_id, trial_names, processed_folder, masks_stack, n_planes, doubling):
    with h5py.File(hdf5_file_path, 'w') as f:
        exp_grp = f.create_group(sample_id)
        
        all_traces, all_labels, all_planes, all_trials, all_odors, all_centroids = [], [], [], [], [], []
        
        for trial_idx, trial_path in enumerate(trial_names):
            movie_path = os.path.join(processed_folder, f"motion_corrected_{trial_path}")
            movie = imread(movie_path)
            print(f"Processing trial {trial_idx + 1}/{len(trial_names)}, shape: {movie.shape}")

            trial_info = trial_path.split('_')
            trial_num = trial_info[5][1:]
            odor_full = trial_info[6][2:]
            odor = odor_full[2:] if odor_full.startswith('o') else odor_full

            for plane in range(n_planes * doubling):
                plane_movie = movie[plane]
                mask = masks_stack[plane, trial_idx, :, :]

                for label in np.unique(mask):
                    if label != 0:
                        label_mask = mask == label
                        fluorescence_values = plane_movie[:, label_mask].mean(axis=1)
                        
                        all_traces.append(fluorescence_values)
                        all_labels.append(label)
                        all_planes.append(plane)
                        all_trials.append(trial_num)
                        all_odors.append(odor)

                        y, x = np.where(label_mask)
                        centroid = (np.mean(y), np.mean(x))
                        all_centroids.append(centroid)

        # Save data in HDF5 file
        exp_grp.create_dataset('raw_traces', data=np.array(all_traces))
        exp_grp.create_dataset('lm_plane_labels', data=np.array(all_labels))
        exp_grp.create_dataset('plane_nr', data=np.array(all_planes))
        exp_grp.create_dataset('trial_nr', data=np.array(all_trials, dtype='S'))
        exp_grp.create_dataset('odor', data=np.array(all_odors, dtype='S'))
        exp_grp.create_dataset('lm_plane_centroids', data=np.array(all_centroids))

        # Create a mapping group
        mapping_grp = exp_grp.create_group('cell_mapping')
        mapping_grp.create_dataset('neuron_ids', data=np.array([f'n{i}' for i in range(1, len(all_labels) + 1)], dtype='S'))
        mapping_grp.create_dataset('lm_plane_labels', data=np.array(all_labels))
        mapping_grp.create_dataset('plane_nr', data=np.array(all_planes))

    print("Fluorescence intensities calculated and saved in HDF5 file.")

# Process and save data
extract_fluorescence_data(hdf5_file_path, sample_id, trial_names, processed_folder, masks_stack, n_planes, doubling)

    
   

20220118_RM0012_124hpf_fP8_f2


<tifffile.TiffFile 'motion_corrected…_o1Ala_001_.tif'> contains no pages


Processing trial 1/24, shape: (0,)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [3]:
 # Plot extracted raw traces
def load_data(hdf5_file_path, sample_id):
    with h5py.File(hdf5_file_path, 'r') as f:
        exp_grp = f[sample_id]
        return {key: exp_grp[key][()] for key in exp_grp.keys() if isinstance(exp_grp[key], h5py.Dataset)}

# Load data
data = load_data(hdf5_file_path, sample_id)
print(data.keys())

In [ ]:
# Plot the first three traces
plt.figure(figsize=(10, 6))
for i in range(3):
    plt.plot(data['raw_traces'][i], label=f'Label {data["lm_plane_labels"][i]}')

plt.title('Fluorescence Intensity Traces')
plt.xlabel('Time (frames)')
plt.ylabel('Fluorescence Intensity')
plt.legend()
plt.show()

In [7]:


# Plot average traces for each odor
odors_name = np.unique(data['odor'])
plt.figure(figsize=(10, 6))
for odor in odors_name:
    plt.plot(data['raw_traces'][data['odor'] == odor].mean(axis=0), label=odor.decode('utf-8'))
plt.legend()
plt.title('Average Fluorescence Intensity Traces by Odor')
plt.xlabel('Time (frames)')
plt.ylabel('Fluorescence Intensity')
plt.show()



In [25]:
import os
import glob
import numpy as np
import pandas as pd
from tifffile import imread
import h5py
import matplotlib.pyplot as plt
from scripts.sample_db import SampleDB

def print_hdf5_structure(item, indent=0):
    for key in item.keys():
        print("  " * indent + str(key))
        if isinstance(item[key], h5py.Group):
            print_hdf5_structure(item[key], indent + 1)
        elif isinstance(item[key], h5py.Dataset):
            print("  " * (indent + 1) + "Shape:", item[key].shape)

# Function to process and save data
def extract_fluorescence_data(hdf5_file_path, sample_id, trial_names, processed_folder, masks_stack, n_planes, doubling):
    all_data = {
        'traces': [],
        'labels': [],
        'planes': [],
        'trials': [],
        'odors': [],
        'centroids': []
    }

    for trial_path in trial_names:
        movie_path = os.path.join(processed_folder, f"motion_corrected_{trial_path}")
        movie = imread(movie_path)
        print(f"Processing trial {trial_path}, shape: {movie.shape}")

        trial_info = trial_path.split('_')
        trial_num = trial_info[5][1:]
        odor_full = trial_info[6][2:]
        odor = odor_full[2:] if odor_full.startswith('o') else odor_full

        for plane in range(n_planes * doubling):
            plane_movie = movie[plane]
            mask = masks_stack[plane % masks_stack.shape[0], :, :]

            for label in np.unique(mask):
                if label != 0:
                    label_mask = mask == label
                    fluorescence_values = plane_movie[:, label_mask].mean(axis=1)
                    
                    all_data['traces'].append(fluorescence_values)
                    all_data['labels'].append(label)
                    all_data['planes'].append(plane)
                    all_data['trials'].append(trial_num)
                    all_data['odors'].append(odor)

                    y, x = np.where(label_mask)
                    centroid = (np.mean(y), np.mean(x))
                    all_data['centroids'].append(centroid)

    # Save data in HDF5 file
    with h5py.File(hdf5_file_path, 'w') as f:
        exp_grp = f.create_group(sample_id)
        
        exp_grp.create_dataset('raw_traces', data=np.array(all_data['traces']))
        exp_grp.create_dataset('lm_plane_labels', data=np.array(all_data['labels']))
        exp_grp.create_dataset('plane_nr', data=np.array(all_data['planes']))
        exp_grp.create_dataset('trial_nr', data=np.array(all_data['trials'], dtype='S'))
        exp_grp.create_dataset('odor', data=np.array(all_data['odors'], dtype='S'))
        exp_grp.create_dataset('lm_plane_centroids', data=np.array(all_data['centroids']))

        # Create a mapping group
        mapping_grp = exp_grp.create_group('cell_mapping')
        mapping_grp.create_dataset('neuron_ids', data=np.array([f'n{i}' for i in range(1, len(all_data['labels']) + 1)], dtype='S'))
        mapping_grp.create_dataset('lm_plane_labels', data=np.array(all_data['labels']))
        mapping_grp.create_dataset('plane_nr', data=np.array(all_data['planes']))

    print("Fluorescence intensities calculated and saved in HDF5 file.")
    print_hdf5_structure(f)

# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Load experiment
sample_id = '20220426_RM0008_130hpf_fP1_f3'  # Replace with your sample ID
exp = sample_db.get_sample(sample_id)
print(f"Processing sample: {exp.sample.id}")

# Making shortcuts of sample parameters/information
trials_path = exp.paths.trials_path
n_planes = exp.params_lm.n_planes
doubling = 2 if exp.params_lm.doubling else 1

# Getting paths of the trial acquisitions
trial_names = os.listdir(os.path.join(trials_path, 'raw'))
processed_folder = os.path.join(trials_path, 'processed')
masks_folder = os.path.join(trials_path, "masks")

# Load the masks
masks_file = glob.glob(os.path.join(masks_folder, f'masks_{exp.sample.id}_*.tif'))[0]
masks_stack = imread(masks_file)

# Create folder for saving fluorescence data
traces_folder = os.path.join(trials_path, "traces")
os.makedirs(traces_folder, exist_ok=True)

hdf5_file_path = os.path.join(traces_folder, f'{exp.sample.id}_fluorescence_data.h5')

# Process and save data
extract_fluorescence_data(hdf5_file_path, sample_id, trial_names, processed_folder, masks_stack, n_planes, doubling)

# Load data and create plots
def load_data(hdf5_file_path, sample_id):
    with h5py.File(hdf5_file_path, 'r') as f:
        exp_grp = f[sample_id]
        return {key: exp_grp[key][()] for key in exp_grp.keys() if isinstance(exp_grp[key], h5py.Dataset)}

data = load_data(hdf5_file_path, sample_id)

# Plot the first three traces
plt.figure(figsize=(10, 6))
for i in range(3):
    plt.plot(data['raw_traces'][i], label=f'Label {data["lm_plane_labels"][i]}')

plt.title('Fluorescence Intensity Traces')
plt.xlabel('Time (frames)')
plt.ylabel('Fluorescence Intensity')
plt.legend()
plt.show()

# Plot average traces for each odor
odors_name = np.unique(data['odor'])
plt.figure(figsize=(10, 6))
for odor in odors_name:
    plt.plot(data['raw_traces'][data['odor'] == odor].mean(axis=0), label=odor.decode('utf-8'))
plt.legend()
plt.title('Average Fluorescence Intensity Traces by Odor')
plt.xlabel('Time (frames)')
plt.ylabel('Fluorescence Intensity')

# Save the plot
report_folder = os.path.join(exp.paths.root_path, "report")
os.makedirs(report_folder, exist_ok=True)
plt.savefig(os.path.join(report_folder, f'10_extract_lm_traces_average_traces_{exp.sample.id}.png'))
plt.close()

# Update sample database
sample_db.update_sample_field(sample_id, '10_extract_lm_traces', True)
sample_db.save(db_path)

print(f"LM traces extraction completed for sample: {exp.sample.id}")

Processing sample: 20220426_RM0008_130hpf_fP1_f3


<tifffile.TiffFile 'motion_corrected…_o1Ala_001_.tif'> contains no pages


Processing trial 20220426_RM0008_130hpf_fP1_f3_t1_o1Ala_001_.tif, shape: (0,)


IndexError: index 0 is out of bounds for axis 0 with size 0